<img src="../resources/cropped-SummerWorkshop_Header.png">  

<h1 align="center">Brain Observatory Workshop SWDB 2018 </h1> 
<h3 align="center">Monday, August 20, 2018</h3> 

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

<p>This notebook will introduce you to the Allen Brain Observatory dataset and SDK functions. 

<p>As an example use case we will look at signal correlations between all the neurons in a given experiment and see whether this is related to the distance between the cells.
</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<h2>Drive path</h2>

<p>First things first - let's make sure your drive path is set up appropriately. You will want to un-comment only one of these options depending on which platform you are using. You also may have to tweak the option for your specific set-up (e.g. change the drive letter for Windows).
</p>
</div>

In [ ]:
#make sure your drive path is correct! 
# macOS/OS X
# drive_path = '/Volumes/Brain2018/visual_coding_2p/'

# Windows (a good guess)
# drive_path = 'e:/visual_coding_2p'

# Linux (will vary; the following is possibly what Ubuntu will do)
# drive_path = '/media/Brain2018/visual_coding_2p'

# AWS
drive_path = '/data/allen-brain-observatory/visual-coding-2p'

In [ ]:
# We need to import these modules to get started
import numpy as np
import pandas as pd
import os
import sys
import h5py

import matplotlib.pyplot as plt
%matplotlib inline

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>The main entry point is the `BrainObservatoryCache` class.  This class is responsible for downloading any requested data or metadata as needed and storing it in well known locations.  For this workshop, all of the data has been preloaded onto the hard drives you have received, and is available on AWS.

<p>We begin by importing the `BrainObservatoryCache` class and instantiating it.

<p>`manifest_path` is a path to the manifest file.  We will use the manifest file preloaded onto your Workshop hard drives.  Make sure that `drive_path` is set correctly for your platform.  (See the first cell in this notebook.)
</div>


In [ ]:
from allensdk.core.brain_observatory_cache import BrainObservatoryCache

manifest_file = os.path.join(drive_path,'manifest.json')
print(manifest_file)

boc = BrainObservatoryCache(manifest_file=manifest_file)

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Task 1.1:</b>  Get information about what's in the dataset from BrainObservatoryCache

<p>The following methods for BrainObservatoryCache retrieve the available depths, cre lines, areas, and stimuli.  Notice that these parameters outline the 'data cube'.
</div>

In [ ]:
# Download a list of all targeted areas
targeted_structures = boc.get_all_targeted_structures()
print('all targeted structures: ' + str(targeted_structures))

# Download a list of all imaging depths
depths = boc.get_all_imaging_depths()
print('all imaging depths: ' + str(depths))

# Download a list of all cre driver lines 
cre_lines = boc.get_all_cre_lines()
print('all cre lines: ' + str(cre_lines))

# Download a list of all stimuli
stims = boc.get_all_stimuli()
print('all stimuli: ' + str(stims))

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Task 1.2:</b>  Use tab completion in Jupyter to see what other methods the BrainObservatoryCache has.
</div>

In [ ]:
# Hit the 'tab' key with the cursor just after the '.'
boc.

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<h2>Experiment containers</h2>
<p>The experiment container describes a set of 3 experiment sessions performed at the same location (targeted area and imaging depth) in the same mouse that targets the same set of cells. Each experiment container has a unique ID number.
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Task 1.3:</b>  Choose a visual area and Cre line from the lists above to examine in the rest of the notebook
</div>

In [ ]:
visual_area = 'VISp'
cre_line ='Cux2-CreERT2'

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Task 1.4:</b>  Get the list of all the experiment containers for that area and Cre line combination.
</div>

In [ ]:
exps = boc.get_experiment_containers(targeted_structures=[visual_area], cre_lines=[cre_line])

In [ ]:
#make a pandas DataFrame to see what information we have on these experiments
pd.DataFrame(exps)

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Task 1.5:</b>  Pick a random experiment from this selection
</div>

In [ ]:
expt_container_id = np.random.choice(exps)['id']

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Task 1.6:</b> Get information about all of the experiment <strong>sessions</strong> in your experiment <strong>container</strong>.  This is accomplished with the `get_ophys_experiments` method.  
</div>

In [ ]:
expt_session_info = boc.get_ophys_experiments(experiment_container_ids=[expt_container_id])
print(expt_session_info)
print(len(expt_session_info))

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

<p>`get_experiment_containers` returns a list of dictionaries that contain information about <b>experiment containers</b>.

<p>`get_ophys_experiments` returns a list of dictionaries that contain information about <b>experiment sessions</b>.  Here we are using keyword arguments to return just those experiment sessions that belong to our experiment container. We could have used targeted structures and Cre lines to find all the experiment sessions in our chosen location.
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Task 1.7:</b> Create a dataframe of the experiment sessions in your chosen container.  
</div>

In [ ]:
expt_session_info_df = pd.DataFrame(expt_session_info)
expt_session_info_df

How many sessions are in your experiment container? What is different about each session? 

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Exercise 1.8:</b>  Find the session id for the session that has Natural Scenes as a stimulus for your chosen experiment container.  Save this as `session_id`. (Hint: you can either know which session type contains the Natural Scenes stimulus, or you can use the stimulus to select the experiment session)
</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>We will be using the session you have recorded in `session_id` for much of the remainder of this notebook.

<h2>The Dataset Object</h2>
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Task 2.1:</b>  Create a data_set object for this experiment session.

The data_set object contains methods and info for a single experiment session (one of the 3 in the experiment container)
</div>

In [ ]:
# make a data_set object
data_set = boc.get_ophys_experiment_data(ophys_experiment_id=session_id)

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Task 2.2:</b> Use either `dir` or tab-completion to find out what methods the new `data_set` object has.
</div>

In [ ]:
data_set.

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

<p>Using the methods you find, perform the following exercises.
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Exercise 2.3:</b> Get the metadata for your data set. How old was the mouse in this experiment?  Was it male or female?  
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Exercise 2.4:</b> Get the max projection image for your data set. Display it.
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Exercise 2.5:</b> Get the roi mask for all the cells.  (Hint:  There are two methods that return roi masks.  In one of them masks are returned as lists of python objects.  What methods do they have?  What is the type of this object?)  What is the size and shape of the mask? How many cells are in this experiment session?
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Exercise 2.6:</b> Pick one cell (at random) and plot the mask overlayed on the max projection.  (Hint:  imshow has an optional parameter called `alpha`.) What is the structure of the mask - how can you find the position of the mask?
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Exercise 2.7:</b> Calculate the centroid of the ROI mask. Verify that you've calculated this properly by plotting it on top of the mask
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Exercise 2.8:</b> Find the centroid of each roi mask and calculate the distance between all the ROIs
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Exercise 2.9:</b> Plot the distribution of distances. Only count each distance once! (Hint: numpy has functions to select the upper or lower triangle of an array. Look for triu, or triu_indices. Be sure to eliminate the diagonal itself)
</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<h2>The Analysis Object</h2>


<p>The analysis objects summarize the trial data for a stimulus type and provide convenient DataFrame objects.  
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Task 3.1:</b>  Import the `NaturalScenes` object and instatiate it with `data_set`. Explore the contents and functions of this object.
</div>

In [ ]:
from allensdk.brain_observatory.natural_scenes import NaturalScenes
ns = NaturalScenes(data_set) 

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
To save time, many of the items that are computed (or computable) in the analysis object are stored in <u>analysis files</u> that we have made available to you.  For the next task we will acess the signal correlations from this analysis file, as this is a particularly slow computation.
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Task 3.2:</b>  Get the "signal correlations" of the responses to natural scenes. This can be computed in from the analysis object, using ns.get_signal_correlation(), but this is very slow. Instead we will extract the result from the analysis file in the steps below.
</div>

In [ ]:
ns = boc.get_ophys_experiment_analysis(ophys_experiment_id=session_id, stimulus_type='natural_scenes')
sc = ns.signal_correlation

In [ ]:
print(sc.shape)

In [ ]:
plt.imshow(sc, cmap='viridis', interpolation='none')
plt.xlabel("Cell #")
plt.ylabel("Cell #")
plt.title("Signal Correlations")
plt.colorbar()

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Exercise 3.3:</b> Explore the other items that are available in the analysis object.
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Exercise 3.4:</b> Plot the signal correlation vs distance for all pairs of cells.
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Exercise 3.5:</b> Find the most signal-correlated pair of neurons in this experiment and plot their ROIs.
</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

<p>We want to look at the trial-by-trials responses of these cells to their preferred stimuli. To do this we're going to pull out the responses from our analysis object.

<p>sweep_response is a DataFrame that contains the dF/F response of each cell during each stimulus trial. It shares its index with stim_table. Each cell contains a timeseries that extends from 1 second prior to the start of the trial to 1 second after the end of the trial. The sweep_response table is organized as cells (columns) for each sweep (rows)

<p>mean_sweep_response provides the time-averaged dF/F for each trial.

<p>The stimulus_table object contains information about the stimulus. Use data_set.get_stimulus_table('natural_scenes') to get this object. What does it contain?
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Task 3.6:</b> Get the mean sweep responses of your two cells to natural images from the analysis file. What shape is this?
</div>

In [ ]:
mean_sweep_response = ns.mean_sweep_response
mean_sweep_response.shape

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Exercise 3.7:</b> Get the natural scenes stimulus table. What shape is it? What are its columns?
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Exercise 3.8:</b> Use the stimulus table to get the mean sweep responses to natural image 10. Plot your two cells' mean sweep responses to it.
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Exercise 3.9:</b> Get the trial-averaged mean sweep responses to each natural scene for your two cells. (Hint: in addition to using mean_sweep_response and the stimulus table, try checking ns.response.)
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Exercise 3.10:</b> Plot the neurons' preferred images. (Hint: get the stimulus_template object from the data_set. What shape is it?) (Hint2: pay attention to the image indices and the frame labels in the stimulus table)
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Exercise 3.11:</b> Plot the two neurons' trial-by-trial sweep responses to their preferred images. 
</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<h2>Mouse Behavior</h2>


<p>The data_set and analysis objects also contain information about the mouse's running speed and eye tracking. (Not all experiments have eye tracking information.) The running speed, eye position and pupil size can be extracted from the data_set object using appropriately named functions. The running speed has been analyzed the same way as each fluorescent trace, and contained in the final column of the sweep_response and mean_sweep_response dataframes named "dx". 
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Exercise 4.1:</b> Plot the mouse's running speed over the whole experiment and on the same trials you looked at responses of above.
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Exercise 4.2:</b> Check if your experiment has information about the mouse's pupil diameter. If so, plot it on the sweeps of each neuron's preferred image.
</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<h2>The Cell Specimens Object</h2>


<p>The cell specimens object contains summary information about all cells in the brain observatory dataset. Let's find your two cells in it and compare them! First, load the cell_specimens object and see what types of information it contains about each cell.
</div>

In [ ]:
cell_specimens = pd.DataFrame(boc.get_cell_specimens())
print(cell_specimens.keys())

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Exercise 5.1:</b> Find your two cells' cell specimen ids in the data_set object.
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Exercise 5.2:</b> Get the cell_specimens information for just your two cells. (Hint: when you create the cell_specimens object, you can filter by cell specimen id.) What is each of their response reliability for natural scenes? How does it compare to their reliability for static gratings?
</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">


<h2>Project Ideas</h2>
<p>Here are some ideas to get you started in thinking about your projects.


<li> How do cells' responses differ across regions, layers, and Cre lines?  How best can these differences be captured?
    <li> What is the distribution of feature responses?  How does preferred orientation, say, vary across regions, layers, and Cre lines?
    <li> Do the responses to one type of stimulus allow us to predict the responses to a different type?  Are grating responses consistent with natural image responses?
    <li> Can you distinguish \"simple\" and \"complex\" cells in the dataset?  What is the right model or metric to use?
    <li> Characterize the cross correlations (both \"noise\" and \"signal\" correlations) in the data set.  Can you model this variability?
    <li> Develop models of stimulus response that control for running speed or include temporal dynamics.
    <li> How well can you identify the stimulus category given the activity of a set of neurons within an experiment, i.e. can you \"decode\" the stimulus?  What is the best way to do this?  What features are necessary?  Can you identify cells that carry \"more\" information about stimuli?
    <li> What population metrics are useful for describing the data?  Can you model the population activity?
    <li> What is the best way to visualize the activity of many cells in an experiment?  Is there a useful dimensional reduction that can help you?

</div>